# San Diego Burrito Analytics: Coordinates

Determine the longitude and latitude for each restaurant based on its address

### Default imports

In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
sns.set_style("white")

### Load data

In [2]:
import util2
df, dfRestaurants, dfIngredients = util2.load_burritos()
N = df.shape[0]

### Process 

In [3]:
dfRestaurants=dfRestaurants.reset_index().drop('index',axis=1)

In [4]:
dfRestaurants

,Location,Neighborhood,Address,URL,Yelp,Google,Chips
0,Donato's taco shop,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,False
1,Oscar's Mexican food,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,False
2,Pollos Maria,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,True
3,Nico's Taco Shop,Carmel Valley,3860 Valley Centre Dr #404,http://www.yelp.com/biz/nicos-taco-shop-san-diego,3.0,2.9,False
4,Los Primos Mexican Food,UTC,7770 Regents Rd,http://www.primosmex.com/,3.0,3.7,True
5,JV's Mexican Food,Morena,1112 Morena Blvd,http://jvsmexfood.com/,4.0,4.1,False
6,Tony's Fresh Mexican Food,Miramar,7122 Miramar Rd,https://www.yelp.com/biz/tonys-fresh-mexican-f...,3.0,4.1,True
7,Taco Stand,La Jolla,621 Pearl St,http://letstaco.com/,4.5,4.4,False
8,Graciela's Taco Shop,Bonita,5047 Central Ave,https://www.yelp.com/biz/gracielas-taco-shop-b...,4.0,4.5,False
9,Rigoberto's Taco Shop La Jolla,La Jolla,7345 La Jolla Blvd,https://www.yelp.com/biz/rigobertos-la-jolla-2,2.5,4.0,False


### Process Cali burrito data: Averages for each restaurant

In [5]:
dfAvg = df.groupby('Location').agg({'Cost': np.mean,'Volume': np.mean,'Hunger': np.mean,
                                'Tortilla': np.mean,'Temp': np.mean,'Meat': np.mean,
                                'Fillings': np.mean,'Meat:filling': np.mean,'Uniformity': np.mean,
                                'Salsa': np.mean,'Synergy': np.mean,'Wrap': np.mean,
                                'overall': np.mean, 'Location':np.size})
dfAvg.rename(columns={'Location': 'N'}, inplace=True)
dfAvg['Location'] = list(dfAvg.index)

In [6]:
# Calculate latitutude and longitude for each city
import geocoder
addresses =  dfRestaurants['Address'] + ', ' +  dfRestaurants['Neighborhood'] + ', San Diego, CA' # dfRestaurants['Address'] + ', San Diego, CA'
lats = np.zeros(len(addresses))
longs = np.zeros(len(addresses))
for i, address in enumerate(addresses):
    g = geocoder.google(address)
    Ntries = 1
    while g.latlng ==[]:
        g = geocoder.google(address)
        Ntries +=1
        print 'try again: ' + address
        if Ntries >= 5:
            if 'Marshall College' in address:
                address = '9500 Gilman Drive, La Jolla, CA'
                g = geocoder.google(address)
                Ntries = 1
                while g.latlng ==[]:
                    g = geocoder.google(address)
                    Ntries +=1
                    print 'try again: ' + address
                    if Ntries >= 5:
                        raise ValueError('Address not found: ' + address)
            else:
                raise ValueError('Address not found: ' + address)
    lats[i], longs[i] = g.latlng
    
# # Check for nonsense lats and longs
if sum(np.logical_or(lats>34,lats<32)):
    raise ValueError('Address not in san diego')
if sum(np.logical_or(longs<-118,longs>-117)):
    raise ValueError('Address not in san diego')

try again: Marshall College, La Jolla, San Diego, CA
try again: Marshall College, La Jolla, San Diego, CA
try again: Marshall College, La Jolla, San Diego, CA
try again: Marshall College, La Jolla, San Diego, CA
try again: 9500 Gilman Dr, UTC, San Diego, CA
try again: 3202 Mission Blvd, Mission Beach, San Diego, CA


In [7]:
# Incorporate lats and longs into restaurants data
dfRestaurants['Latitude'] = lats
dfRestaurants['Longitude'] = longs
# Merge restaurant data with burrito data
dfTableau = pd.merge(dfRestaurants,dfAvg,on='Location')

In [8]:
dfTableau.head()

,Location,Neighborhood,Address,URL,Yelp,Google,Chips,Latitude,Longitude,Fillings,...,N,Tortilla,Synergy,Uniformity,Hunger,Meat:filling,Volume,Cost,Wrap,Salsa
0,Donato's taco shop,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,False,32.878996,-117.168846,3.500000,...,1,3.000000,4.000000,4.000000,3.000000,4.000000,NaN,6.490000,4.000000,4.000
1,Oscar's Mexican food,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,False,33.145203,-117.200290,2.833333,...,3,2.666667,3.166667,4.333333,2.333333,3.500000,NaN,5.183333,5.000000,3.500
2,Pollos Maria,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,True,33.161188,-117.344319,3.625000,...,4,3.500000,4.250000,3.500000,3.625000,3.375000,0.615000,6.765000,2.500000,2.625
3,Nico's Taco Shop,Carmel Valley,3860 Valley Centre Dr #404,http://www.yelp.com/biz/nicos-taco-shop-san-diego,3.0,2.9,False,32.939136,-117.232163,2.250000,...,2,2.250000,2.250000,3.000000,2.750000,2.750000,NaN,7.090000,3.000000,NaN
4,Los Primos Mexican Food,UTC,7770 Regents Rd,http://www.primosmex.com/,3.0,3.7,True,32.862750,-117.224335,3.090909,...,11,3.363636,2.545455,2.750000,3.318182,1.909091,0.761429,7.077273,3.454545,3.000


In [9]:
dfTableau.to_csv('df_burrito_tableau.csv')